# 实验3：LeNet-5 网络的构建
## 一、实验介绍
以 MNIST 数据集为对象，利用 Pytorch进行 LeNet-5 模型设计、数据加载、损
失函数及优化器定义，评估模型的性能。

通过之前几节，我们学习了构建一个完整卷积神经网络的所需组件。
回想一下，之前我们将softmax回归模型（`sec_softmax_scratch`）和多层感知机模型（`sec_mlp_scratch`）应用于Fashion-MNIST数据集中的服装图片。
为了能够应用softmax回归和多层感知机，我们首先将每个大小为$28\times28$的图像展平为一个784维的固定长度的一维向量，然后用全连接层对其进行处理。
- 而现在，我们已经掌握了卷积层的处理方法，我们可以在图像中保留空间结构。
- 同时，用卷积层代替全连接层的另一个好处是：模型更简洁、所需的参数更少。

本节将介绍LeNet，它是最早发布的卷积神经网络之一，因其在计算机视觉任务中的高效性能而受到广泛关注。
这个模型是由AT&T贝尔实验室的研究员Yann LeCun在1989年提出的（并以其命名），目的是识别图像`LeCun.Bottou.Bengio.ea.1998`中的手写数字。
当时，Yann LeCun发表了第一篇通过反向传播成功训练卷积神经网络的研究，这项工作代表了十多年来神经网络研究开发的成果。

当时，LeNet取得了与支持向量机（support vector machines）性能相媲美的成果，成为监督学习的主流方法。
LeNet被广泛用于自动取款机（ATM）机中，帮助识别处理支票的数字。
时至今日，一些自动取款机仍在运行Yann LeCun和他的同事Leon Bottou在上世纪90年代写的代码呢！

## 二、实验要求
理解常用 Pytorch实现卷积神经网络的流程，定义数据加载器、损失函数和优化器，构建完整的训练流程。
## 三、实验内容

### 3.1 导入所需的依赖包

In [34]:
import torch
import torch.nn
import torch.optim
import torchvision

import torch.utils.data 
import matplotlib.pyplot

### 3.2 数据预处理
在概率论的角度来看，Normalize 里的 mean 和 std 分别对应“期望”（Arithmetic Mean / Expectation）和“标准差”（Standard Deviation）。这两者是分布最常见的两个刻画参数：  
- mean（期望）描述了数据分布的集中趋势；  
- std（标准差）则描述了分布的离散程度。  

至于为什么在 PyTorch / torchvision 中要传递元组（tuple）而不是单个数值，这是因为在图像处理时通常有多个通道（例如 RGB 图像有 3 个通道），需要对每个通道各自进行正则化：  
$$
\text{output}[c] = \frac{\text{input}[c] - \text{mean}[c]}{\text{std}[c]}
$$
因此，mean 和 std 都需要以元组的形式提供多个通道对应的参数（如：(mean_R, mean_G, mean_B), (std_R, std_G, std_B)）。如果图像只有单通道，也可以只传递包含一个元素的元组。

In [72]:
import torch.utils.data.dataloader
import torchvision.transforms

transform_calling_function = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.13, ), (0.30, ))
])


dataset_train = torchvision.datasets.MNIST(
    root='./data',
    train=True,
    download=True,
    transform=transform_calling_function
)

dataset_test = torchvision.datasets.MNIST(
    root='./data',
    train=False,
    download=True,
    transform=transform_calling_function
)

train_data_loader = torch.utils.data.DataLoader(
    dataset=dataset_train,
    shuffle=False,
    batch_size=1000
)

test_data_loader = torch.utils.data.DataLoader(
    dataset=dataset_test,
    shuffle=False,
    batch_size=64
)

### 3.3 定义LeNet类

总体来看，(**LeNet（LeNet-5）由两个部分组成：**)(卷积编码器和全连接层密集块)

* 卷积编码器：由两个卷积层组成;
* 全连接层密集块：由三个全连接层组成。

该架构如下所示。

![LeNet中的数据流。输入是手写数字，输出为10种可能结果的概率。](../../pytorch/img/lenet.svg)

每个卷积块中的基本单元是一个卷积层、一个sigmoid激活函数和平均汇聚层。请注意，虽然ReLU和最大汇聚层更有效，但它们在20世纪90年代还没有出现。每个卷积层使用$5\times 5$卷积核和一个sigmoid激活函数。这些层将输入映射到多个二维特征输出，通常同时增加通道的数量。第一卷积层有6个输出通道，而第二个卷积层有16个输出通道。每个$2\times2$池操作（步幅2）通过空间下采样将维数减少4倍。卷积的输出形状由批量大小、通道数、高度、宽度决定。

为了将卷积块的输出传递给稠密块，我们必须在小批量中展平每个样本。换言之，我们将这个四维输入转换成全连接层所期望的二维输入。这里的二维表示的第一个维度索引小批量中的样本，第二个维度给出每个样本的平面向量表示。LeNet的稠密块有三个全连接层，分别有120、84和10个输出。因为我们在执行分类任务，所以输出层的10维对应于最后输出结果的数量。

通过下面的LeNet代码，可以看出用深度学习框架实现此类模型非常简单。我们只需要实例化一个`Sequential`块并将需要的层连接在一起。




**输入图像**：
- 经典的输入尺寸是 `(1, 32, 32)` 单通道灰度图像 (MNIST)，但你也可以适当调整。

**结构顺序**：

1. **卷积层（Conv1）**：
   - 输入通道：1
   - 输出通道：6
   - 卷积核尺寸：5×5
   - 激活函数：Sigmoid（经典LeNet中为sigmoid，现通常用ReLU）

2. **平均池化层（AvgPool1）**：
   - 核尺寸：2×2
   - 步长：2

3. **卷积层（Conv2）**：
   - 输入通道：6
   - 输出通道：16
   - 卷积核尺寸：5×5
   - 激活函数：Sigmoid（经典）或ReLU

4. **平均池化层（AvgPool2）**：
   - 核尺寸：2×2
   - 步长：2

5. **Flatten 展平**：
   - 把特征图展平成一维向量。

6. **全连接层（FC1）**：
   - 输入维度：400（若输入32×32图像，经上述层变换最终变成16通道×5×5尺寸=400个神经元）
   - 输出维度：120
   - 激活函数：Sigmoid（经典）或ReLU

7. **全连接层（FC2）**：
   - 输入维度：120
   - 输出维度：84
   - 激活函数：Sigmoid（经典）或ReLU

8. **全连接输出层（FC3）**：
   - 输入维度：84
   - 输出维度：10（用于MNIST的10分类）
   - 激活函数：一般不加，直接输出Logits

**常见激活函数选择说明**：
- **经典论文版本（LeNet-5）**：使用Sigmoid激活函数
- **现代常用实践**：使用ReLU替代Sigmoid以提高训练效率

**整体网络结构示意图**：

```
Input → Conv(1→6, 5×5) → Pool(2×2) → Conv(6→16, 5×5) → Pool(2×2) → Flatten → FC(400→120) → FC(120→84) → FC(84→10) → Output
```

请根据以上提示信息，在不参考任何代码示例的情况下，默写PyTorch代码实现LeNet。

In [59]:
import torch.nn
class My_LeNet(torch.nn.Module): # 继承
    def __init__(self, *args, **kwargs):
        # python2的写法：
        # super(My_LeNet, self).__init__(*args, **kwargs) # 初始化父类，super是父类（超类）的意思
        # python3的写法：
        super().__init__(*args, **kwargs) # 初始化父类，super是父类（超类）的意思
        self.convolution_layer1 = torch.nn.Conv2d(
            in_channels=1, 
            out_channels=6, # 一共六层参数
            kernel_size=(5, 5), 
            padding=2 # padding成32*32 
        )
        self.avg_pooling1 = torch.nn.AvgPool2d( # 缩成 16*16 还是六层参数
            kernel_size=(2, 2), 
            stride=2
        )

        # 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16
        self.convolution_layer2 = torch.nn.Conv2d(
            in_channels=6, 
            out_channels=16, # 我有16个卷积核，每个卷积核都会作用在6个输入平面上，最后输出16层
            kernel_size=(5, 5)
        ) # 输出尺寸：（16-5+1）= 12 * 12 * 16

        self.avg_pooling2 = torch.nn.AvgPool2d(
            kernel_size=(2, 2), 
            stride=2
        ) # 输出尺寸：6 * 6 * 16

        # ============== 重点：自动推导in_features ===============
        with torch.no_grad():
            fake_input = torch.zeros((1, 1, 28, 28))  # 假设输入图片为(1, 1, 32, 32)
            output = self.convolution_layer1(fake_input)
            output = self.avg_pooling1(output)
            output = self.convolution_layer2(output)
            output = self.avg_pooling2(output)
            output_dim = output.view(1, -1).shape[1]  # 自动推导尺寸

        # ============== 推导完成 ===========================

        self.fully_connected_layer1 = torch.nn.Linear(in_features=output_dim, out_features=120, bias=True)
        self.fully_connected_layer2 = torch.nn.Linear(in_features=120, out_features=84, bias=True)
        self.fully_connected_layer3 = torch.nn.Linear(in_features=84, out_features=10, bias=True)
        
        # 这里只创建了一个激活函数，因为激活函数不用训练。。。
        self.activation_func = torch.nn.ReLU()

    def forward(self, input):
        output = self.convolution_layer1(input)
        output = self.activation_func(output)
        output = self.avg_pooling1(output)

        output = self.convolution_layer2(output)
        output = self.activation_func(output)
        output = self.avg_pooling2(output)

        output = output.view(output.size(0), -1) # 展平
        # print("Flattened shape:", output.shape) # 确认一下维度，是否与预期一致
        
        output = self.fully_connected_layer1(output)
        output = self.activation_func(output)
        output = self.fully_connected_layer2(output)
        output = self.activation_func(output)
        output = self.fully_connected_layer3(output)

        return output



### 3.4 初始化模型、损失函数和优化器

其实交叉熵损失可以传入权重，让少数类设置更大权重，重要类别设置更大权重
样本量不平衡时的损失调整
例如：在100张图片中

- 类别0: 50张 -> weight=1.0
- 类别1: 25张 -> weight=2.0
- 类别2: 25张 -> weight=2.0

In [60]:
import torch
device = torch.device

if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
print(f"training on {device.type}")

model = My_LeNet().to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)


training on cuda


### 3.5 训练模型

In [78]:
import torch

class O:
    def o():
        print("尊嘟假嘟")

O.o()

import os

# 定义保存路径
path_to_saved_parameters = "./parameters"

# 创建保存目录
if not os.path.exists(path_to_saved_parameters):
    os.makedirs(path_to_saved_parameters)



def train(loader = train_data_loader, 
          epochs = 0xA):
    model.train() # Set the module in training mode
    for current_epoch in range(epochs):
        # 统计一个epoch的总loss，最后除以训练集的size
        epoch_loss = 0
    
        for img, true_label in loader:
            # Move input data to the same device as model
            img = img.to(device)
            true_label = true_label.to(device)
            
            optimizer.zero_grad() # 优化器里保存了上次的loss的梯度
            output_label = model.forward(img)
            loss = criterion(output_label, true_label)
            loss.backward() # 计算完loss对每个参数的梯度后，将梯度存储到参数的.grad属性里
            optimizer.step() # 再由优化器进行优化
            epoch_loss += loss.item()
        # 保存参数
        torch.save({
            'epoch': current_epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss,
        }, f"{path_to_saved_parameters}/checkpoint_epoch_{current_epoch}.pt")

        print(f"Epoch[{current_epoch}/{epochs}], loss:{epoch_loss/len(loader)}")

train()


尊嘟假嘟
Epoch[0/10], loss:0.28058180287480355
Epoch[1/10], loss:0.27254513079921405
Epoch[2/10], loss:0.2648617774248123
Epoch[3/10], loss:0.2575187755127748
Epoch[4/10], loss:0.25051901638507845
Epoch[5/10], loss:0.24378356399635473
Epoch[6/10], loss:0.23733534601827463
Epoch[7/10], loss:0.2311368878930807
Epoch[8/10], loss:0.22521438238521416
Epoch[9/10], loss:0.21951047368347645


### 3.6 评估模型

In [80]:
def evaluate(model = model, 
             loader = test_data_loader,
             load_path = "./parameters/checkpoint_epoch_5.pt"):
    model.eval()

    # 加载模型参数
    checkpoint = torch.load(load_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy: .2f}%")

evaluate()


C:\Users\25891\AppData\Local\Temp\ipykernel_33964\1014593513.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(load_path)


Test Accuracy:  93.26%


In [81]:
model.parameters

<bound method Module.parameters of My_LeNet(
  (convolution_layer1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (avg_pooling1): AvgPool2d(kernel_size=(2, 2), stride=2, padding=0)
  (convolution_layer2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (avg_pooling2): AvgPool2d(kernel_size=(2, 2), stride=2, padding=0)
  (fully_connected_layer1): Linear(in_features=400, out_features=120, bias=True)
  (fully_connected_layer2): Linear(in_features=120, out_features=84, bias=True)
  (fully_connected_layer3): Linear(in_features=84, out_features=10, bias=True)
  (activation_func): ReLU()
)>